# Importing Libraries 

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import HTML

# Uploading Data

In [7]:
files = ['AIRPORTS',
'AREA',
'BUDGET',
'continent',
'CURACCT',
'DEBTFOREIGNERS',
'EDUCATIONSPEND',
'electricconsupmtion',
'emissions',
'EXPORTS',
'fossilelectricity',
'GDP',
'GDPGROWTH',
'GDPPERCAP',
'hydroelectricity',
'IMPORTS',
'INDUSTPRODGR',
'internetusers',
'LIFEXP',
'MARMERCHANT',
'MIGRATION',
'MILIEXPENDITURE',
'naturalgasconsumption',
'nuclearelectricity',
'otherrenewableelectricity',
'petroleumconsumption',
'POPGROWTH',
'POPULATION',
'PUBLICDEBT',
'RAILWAYS',
'ROADWAYS',
'UNEMPLOYMENT',
'WATERWAYS',
'WORKFORCE']

In [8]:
pd_dict = {}
for file in files:
    pd_dict[file] = pd.read_csv('{}.csv'.format(file))

In [9]:
pd_dict['AIRPORTS']

,Unnamed: 0,COUNTRY,AIRPORTS,YEAR
0,1,United States,"13,513",2013
1,2,Brazil,"4,093",2013
2,3,Mexico,"1,714",2013
3,4,Canada,"1,467",2013
4,5,Russia,"1,218",2013
...,...,...,...,...
233,234,Saint Barthelemy,1,2013
234,235,Saint Martin,1,2013
235,236,Sint Maarten,1,2013
236,237,Tuvalu,1,2013


In [10]:
pd_dict['continent']

,continent_name,country_name
0,Asia,"Afghanistan, Islamic Republic of"
1,Europe,"Albania, Republic of"
2,Antarctica,Antarctica (the territory South of 60 deg S)
3,Africa,"Algeria, People's Democratic Republic of"
4,Oceania,American Samoa
...,...,...
252,Africa,"Zambia, Republic of"
253,Oceania,Disputed Territory
254,Asia,Iraq-Saudi Arabia Neutral Zone
255,Asia,United Nations Neutral Zone


In [16]:
base_df = pd_dict['continent']
area_df = pd_dict['AREA']
petrol_df = pd_dict['petroleumconsumption']
users_df = pd_dict['internetusers']
hydro_df = pd_dict['hydroelectricity']
nuclear_df = pd_dict['nuclearelectricity']
otherrenew_df = pd_dict['otherrenewableelectricity']
mili_df = pd_dict['MILIEXPENDITURE']
co2_df = pd_dict['emissions']
mili_df.rename(columns = {'% OF GDP':'mili_exp_%_gdp'}, inplace = True)

In [12]:
for i, row in base_df.iterrows(): 
    row['country_name'] = row['country_name'].split(",")
    row ['country_name'] = row['country_name'][0]

In [17]:
base_df.head()

,continent_name,Country
0,Asia,Afghanistan
1,Europe,Albania
2,Antarctica,Antarctica (the territory South of 60 deg S)
3,Africa,Algeria
4,Oceania,American Samoa


In [18]:
mili_df.head()

,Unnamed: 0,country,mili_exp_%_gdp,YEAR,Unnamed: 4
0,1,Saudi Arabia,8.78,2018,NaN
1,2,Oman,8.17,2018,NaN
2,3,United Arab Emirates,5.70,2016,NaN
3,4,Algeria,5.27,2018,NaN
4,5,Kuwait,5.06,2018,NaN


In [19]:
base_df.rename(columns = {'country_name':'Country'}, inplace = True)
df1 = base_df.merge(area_df, on = 'Country', how = 'outer')

df1.rename(columns = {'Country':'country'}, inplace = True)
df2 = df1.merge(petrol_df, on = 'country', how = 'outer')
df3 = df2.merge(users_df, on = 'country', how = 'outer')
df4 = df3.merge(hydro_df, on = 'country', how = 'outer')
df5 = df4.merge(nuclear_df, on = 'country', how = 'outer')
df6 = df5.merge(otherrenew_df, on = 'country', how = 'outer')
mili_df.rename(columns = {'COUNTRY':'country'}, inplace = True)
df7 = df6.merge(mili_df, on = 'country', how = 'outer')
df8 = df7.merge(co2_df, on = 'country', how = 'outer')


df8.head()


,continent_name,country,Index,Area_sqKm,Unnamed: 0_x,refined_petroleum_consumption_barrels_per_day,year_x,Unnamed: 4_x,Unnamed: 0_y,number_of_internet_users,...,year_y,Unnamed: 4_y,Unnamed: 0_y,mili_exp_%_gdp,YEAR,Unnamed: 4_x,Unnamed: 0,year,carbon_emissions_from_energy_consumption_MT,Unnamed: 4_y
0,Asia,Afghanistan,42.0,"652,230.00",117.0,"35,000",2016.0,est.,NaN,NaN,...,2017.0,est.,116.0,0.99,2018.0,NaN,111.0,2017.0,"9,067,000.00",est.
1,Europe,Albania,145.0,"28,748.00",120.0,"29,000",2016.0,est.,111.0,"2,016,516",...,2017.0,est.,97.0,1.26,2019.0,est.,136.0,2017.0,"4,500,000.00",est.
2,Antarctica,Antarctica (the territory South of 60 deg S),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Africa,Algeria,11.0,"2,381,740.00",37.0,"405,000",2016.0,est.,37.0,"17,291,463",...,2017.0,est.,4.0,5.27,2018.0,NaN,34.0,2017.0,"135,900,000.00",est.
4,Oceania,American Samoa,216.0,224,192.0,"2,375",2016.0,est.,206.0,"17,000",...,2017.0,est.,NaN,NaN,NaN,NaN,189.0,2017.0,"361,100.00",est.


In [20]:
final_df = df8[['continent_name','country','Area_sqKm','refined_petroleum_consumption_barrels_per_day','number_of_internet_users', 'hydro_percentage_of_electricity_production','nuclear_percentage_of_electricity_production', 'other_renewable_percentage_of_electricity_production','mili_exp_%_gdp',"carbon_emissions_from_energy_consumption_MT"]].copy()
final_df

,continent_name,country,Area_sqKm,refined_petroleum_consumption_barrels_per_day,number_of_internet_users,hydro_percentage_of_electricity_production,nuclear_percentage_of_electricity_production,other_renewable_percentage_of_electricity_production,mili_exp_%_gdp,carbon_emissions_from_energy_consumption_MT
0,Asia,Afghanistan,"652,230.00","35,000",NaN,52.0,0.0,4.0,0.99,"9,067,000.00"
1,Europe,Albania,"28,748.00","29,000","2,016,516",95.0,0.0,0.0,1.26,"4,500,000.00"
2,Antarctica,Antarctica (the territory South of 60 deg S),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Africa,Algeria,"2,381,740.00","405,000","17,291,463",1.0,0.0,2.0,5.27,"135,900,000.00"
4,Oceania,American Samoa,224,"2,375","17,000",0.0,0.0,2.0,NaN,"361,100.00"
...,...,...,...,...,...,...,...,...,...,...
295,NaN,Johnston Atoll,2.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN
296,NaN,Howland Island,1.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
297,NaN,Kingman Reef,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
298,NaN,Holy See (Vatican City),0.44,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
final_df['Area_sqKm'] = final_df['Area_sqKm'].str.replace(',','')
final_df['refined_petroleum_consumption_barrels_per_day'] = final_df['refined_petroleum_consumption_barrels_per_day'].str.replace(',', '')
final_df['number_of_internet_users'] = final_df['number_of_internet_users'].str.replace(',', '')
final_df['carbon_emissions_from_energy_consumption_MT'] = final_df['carbon_emissions_from_energy_consumption_MT'].str.replace(',', '')



In [35]:
#Do countries with high carbon emissions burn petrol? 
final_df = final_df.sort_values(by = ['Area_sqKm'], ascending = False)
final_df.dropna(subset=['continent_name', "Area_sqKm"], inplace = True)
final_df.head(n=50)

,continent_name,country,Area_sqKm,refined_petroleum_consumption_barrels_per_day,number_of_internet_users,hydro_percentage_of_electricity_production,nuclear_percentage_of_electricity_production,other_renewable_percentage_of_electricity_production,mili_exp_%_gdp,carbon_emissions_from_energy_consumption_MT
37,North America,Canada,9984670.00,2445000,31770034,56.0,9.0,12.0,1.31,640600000.00
243,North America,United States,9833517.00,19960000,246809221,7.0,9.0,14.0,3.42,5242000000.00
198,Africa,Sao Tome and Principe,964,1000,50000,11.0,0.0,1.0,0.00,148100.00
44,Asia,China,9596960.00,12470000,730723960,18.0,2.0,18.0,1.87,11670000000.00
232,North America,Turks and Caicos Islands,948,1420,NaN,0.0,0.0,0.0,0.00,221800.00
242,Africa,Tanzania,947300.00,72000,6822754,40.0,0.0,6.0,1.21,14570000.00
101,Europe,Hungary,93028.00,167700,7826695,1.0,22.0,13.0,1.21,51280000.00
56,Europe,Cyprus,9251.00,49000,915036,0.0,0.0,15.0,1.57,7720000.00
57,Asia,Cyprus,9251.00,49000,915036,0.0,0.0,15.0,1.57,7720000.00
162,Africa,Nigeria,923768.00,325000,47759904,19.0,0.0,0.0,0.51,104000000.00


In [36]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 201 entries, 37 to 237
Data columns (total 10 columns):
continent_name                                          201 non-null object
country                                                 201 non-null object
Area_sqKm                                               201 non-null object
refined_petroleum_consumption_barrels_per_day           201 non-null object
number_of_internet_users                                197 non-null object
hydro_percentage_of_electricity_production              201 non-null float64
nuclear_percentage_of_electricity_production            201 non-null float64
other_renewable_percentage_of_electricity_production    201 non-null float64
mili_exp_%_gdp                                          201 non-null float64
carbon_emissions_from_energy_consumption_MT             201 non-null object
dtypes: float64(4), object(6)
memory usage: 17.3+ KB


In [37]:
#Do countries with high carbon emissions burn petrol? 
final_df = final_df.sort_values(by = ['number_of_internet_users'], ascending = False)
final_df

,continent_name,country,Area_sqKm,refined_petroleum_consumption_barrels_per_day,number_of_internet_users,hydro_percentage_of_electricity_production,nuclear_percentage_of_electricity_production,other_renewable_percentage_of_electricity_production,mili_exp_%_gdp,carbon_emissions_from_energy_consumption_MT
19,Europe,Belgium,30528.00,648600,9870734,1.0,28.0,36.0,0.93,134700000.00
231,Asia,Turkmenistan,488100.00,160000,951925,0.0,0.0,0.0,0.00,100500000.00
85,Africa,Ghana,238533.00,90000,9328018,42.0,0.0,1.0,0.41,13670000.00
56,Europe,Cyprus,9251.00,49000,915036,0.0,0.0,15.0,1.57,7720000.00
57,Asia,Cyprus,9251.00,49000,915036,0.0,0.0,15.0,1.57,7720000.00
...,...,...,...,...,...,...,...,...,...,...
214,Africa,Sudan,1861484.00,112000,10284260,51.0,0.0,6.0,2.28,16030000.00
232,North America,Turks and Caicos Islands,948,1420,NaN,0.0,0.0,0.0,0.00,221800.00
0,Asia,Afghanistan,652230.00,35000,NaN,52.0,0.0,4.0,0.99,9067000.00
212,Africa,South Sudan,644329.00,8000,NaN,0.0,0.0,1.0,1.33,1224000.00


In [38]:
nan = final_df[final_df['Area_sqKm'].isnull()]
len(nan)

0

In [39]:
final_df.dropna(subset = ['refined_petroleum_consumption_barrels_per_day'], inplace = True)

In [40]:
final_df.isnull().sum()

continent_name                                          0
country                                                 0
Area_sqKm                                               0
refined_petroleum_consumption_barrels_per_day           0
number_of_internet_users                                4
hydro_percentage_of_electricity_production              0
nuclear_percentage_of_electricity_production            0
other_renewable_percentage_of_electricity_production    0
mili_exp_%_gdp                                          0
carbon_emissions_from_energy_consumption_MT             0
dtype: int64

In [41]:
median_exp = final_df['mili_exp_%_gdp'].median()
armed_countries = ['Syria','Qatar']
# final_df[(final_df['mili_exp_%_gdp'].isnull()) & (final_df.country.isin(armed_countries))]['mili_exp_%_gdp'] = median_exp

In [42]:
rows_to_change = list(final_df[(final_df['mili_exp_%_gdp'].isnull()) &
         (final_df.country.isin(armed_countries))].index)

In [43]:
final_df.loc[rows_to_change,'mili_exp_%_gdp'] = median_exp
final_df[final_df.country.isin(armed_countries)]

,continent_name,country,Area_sqKm,refined_petroleum_consumption_barrels_per_day,number_of_internet_users,hydro_percentage_of_electricity_production,nuclear_percentage_of_electricity_production,other_renewable_percentage_of_electricity_production,mili_exp_%_gdp,carbon_emissions_from_energy_consumption_MT
220,Asia,Syria,187437.00,134000,5476850,17.0,0.0,0.0,1.49,27510000.00
184,Asia,Qatar,11586.00,277000,2129360,0.0,0.0,1.0,1.49,114200000.00


In [44]:
#Replacing military expenditure for remaining countries with 0 as no armies
final_df['mili_exp_%_gdp'].fillna(0,inplace = True )

In [45]:
final_df.isnull().sum()

continent_name                                          0
country                                                 0
Area_sqKm                                               0
refined_petroleum_consumption_barrels_per_day           0
number_of_internet_users                                4
hydro_percentage_of_electricity_production              0
nuclear_percentage_of_electricity_production            0
other_renewable_percentage_of_electricity_production    0
mili_exp_%_gdp                                          0
carbon_emissions_from_energy_consumption_MT             0
dtype: int64

In [46]:
nan = final_df[final_df.isna().any(axis=1)]
nan

,continent_name,country,Area_sqKm,refined_petroleum_consumption_barrels_per_day,number_of_internet_users,hydro_percentage_of_electricity_production,nuclear_percentage_of_electricity_production,other_renewable_percentage_of_electricity_production,mili_exp_%_gdp,carbon_emissions_from_energy_consumption_MT
232,North America,Turks and Caicos Islands,948,1420,NaN,0.0,0.0,0.0,0.00,221800.00
0,Asia,Afghanistan,652230.00,35000,NaN,52.0,0.0,4.0,0.99,9067000.00
212,Africa,South Sudan,644329.00,8000,NaN,0.0,0.0,1.0,1.33,1224000.00
213,Africa,Western Sahara,266000.00,1700,NaN,0.0,0.0,0.0,0.00,268400.00


In [47]:
final_df['number_of_internet_users'].median()

2677502.0

In [48]:
median_int = final_df['number_of_internet_users'].median()
internet_users = ['Afghanistan','South Sudan']

In [49]:
int_rows_to_change = list(final_df[(final_df['number_of_internet_users'].isnull()) &
         (final_df.country.isin(internet_users))].index)

In [50]:
final_df.loc[int_rows_to_change,'number_of_internet_users'] = median_int
final_df[final_df.country.isin(internet_users)]

,continent_name,country,Area_sqKm,refined_petroleum_consumption_barrels_per_day,number_of_internet_users,hydro_percentage_of_electricity_production,nuclear_percentage_of_electricity_production,other_renewable_percentage_of_electricity_production,mili_exp_%_gdp,carbon_emissions_from_energy_consumption_MT
0,Asia,Afghanistan,652230.00,35000,2.6775e+06,52.0,0.0,4.0,0.99,9067000.00
212,Africa,South Sudan,644329.00,8000,2.6775e+06,0.0,0.0,1.0,1.33,1224000.00


In [51]:
final_df['number_of_internet_users'].fillna(0,inplace = True )

In [52]:
final_df.isnull().sum()

continent_name                                          0
country                                                 0
Area_sqKm                                               0
refined_petroleum_consumption_barrels_per_day           0
number_of_internet_users                                0
hydro_percentage_of_electricity_production              0
nuclear_percentage_of_electricity_production            0
other_renewable_percentage_of_electricity_production    0
mili_exp_%_gdp                                          0
carbon_emissions_from_energy_consumption_MT             0
dtype: int64

In [53]:
final_df.head()

,continent_name,country,Area_sqKm,refined_petroleum_consumption_barrels_per_day,number_of_internet_users,hydro_percentage_of_electricity_production,nuclear_percentage_of_electricity_production,other_renewable_percentage_of_electricity_production,mili_exp_%_gdp,carbon_emissions_from_energy_consumption_MT
19,Europe,Belgium,30528.00,648600,9870734,1.0,28.0,36.0,0.93,134700000.00
231,Asia,Turkmenistan,488100.00,160000,951925,0.0,0.0,0.0,0.00,100500000.00
85,Africa,Ghana,238533.00,90000,9328018,42.0,0.0,1.0,0.41,13670000.00
56,Europe,Cyprus,9251.00,49000,915036,0.0,0.0,15.0,1.57,7720000.00
57,Asia,Cyprus,9251.00,49000,915036,0.0,0.0,15.0,1.57,7720000.00


In [54]:
final_df['renewable_electricity'] = final_df['hydro_percentage_of_electricity_production'] + final_df['nuclear_percentage_of_electricity_production'] + final_df['other_renewable_percentage_of_electricity_production'] 

In [55]:
continent_df = final_df.pivot_table(index = 'continent_name', values = ['refined_petroleum_consumption_barrels_per_day', 
                                                         'number_of_internet_users','renewable_electricity' 
                                                         ,'carbon_emissions_from_energy_consumption_MT',"mili_exp_%_gdp"], aggfunc = np.mean)



continent_df.reset_index(inplace = True)
continent_df

,continent_name,mili_exp_%_gdp,renewable_electricity
0,Africa,1.433469,37.326531
1,Asia,2.495556,27.955556
2,Europe,1.551915,49.595745
3,North America,0.515161,24.225806
4,Oceania,0.249412,27.588235
5,South America,1.488333,51.416667


In [56]:
continent_df.columns

Index(['continent_name', 'mili_exp_%_gdp', 'renewable_electricity'], dtype='object')

In [57]:
continent_df
pd.options.display.float_format = '{:,.2f}'.format

print (continent_df)


  continent_name  mili_exp_%_gdp  renewable_electricity
0         Africa            1.43                  37.33
1           Asia            2.50                  27.96
2         Europe            1.55                  49.60
3  North America            0.52                  24.23
4        Oceania            0.25                  27.59
5  South America            1.49                  51.42


In [58]:
def color_red(value):

    if value >= 100000000 :
        color = 'red'
    elif value < 100000000:
        color = 'green'
    else:
        color = 'black'

    return 'color: %s'% color

In [59]:
continent_df['mili_exp_%_gdp'] = continent_df['mili_exp_%_gdp'].div(100).round(4)
continent_df['renewable_electricity'] = continent_df['renewable_electricity'].div(100).round(4)

cm = sns.light_palette('grey', as_cmap=True)

In [60]:
cdf = continent_df.style.applymap(color_red, subset=['carbon_emissions_from_energy_consumption_MT'])

cdf = continent_df.style.format({'continent_name': str.upper,
                           'carbon_emissions_from_energy_consumption_MT': '{:20,.0f}',"mili_exp_%_gdp":'{:.2%}',
                          "number_of_internet_users":'{:20,.0f}', "refined_petroleum_consumption_barrels_per_day":'{:20,.0f}',"renewable_electricity":'{:.2%}'})

cdf

,continent_name,mili_exp_%_gdp,renewable_electricity
0,AFRICA,1.43%,37.33%
1,ASIA,2.50%,27.96%
2,EUROPE,1.55%,49.60%
3,NORTH AMERICA,0.52%,24.23%
4,OCEANIA,0.25%,27.59%
5,SOUTH AMERICA,1.49%,51.42%
